In [1]:
import numpy as np
from specializeGraph import *
from scipy import linalg as la

### Adding 1 to Diagonal of laplacian of SCC
We find that if we add 1 anywhere to the diagonal of a strongly connected laplacian, it becomes invertible. Amazing!

In [3]:
howMany = 200
n = 4
for j in range(howMany):
    G = randomGraph(n,base=0)
    if nx.is_strongly_connected(nx.DiGraph(G)):
        i = np.random.randint(0,n)
        D = np.zeros((n,n))
        D[i,i] = 1
        if np.isclose(la.det(laplacian(G)+D),0):
            drawGraph(G)
            print(la.eig(laplacian(G))[1])

Question: Is there a pattern to the inverse?

### Find a graph where adding 1 to a diagonal decreases the spectrum

In [4]:
def decrDiagLap(n):
    for i in range(200):
        G = randomGraph(n,stronglyCon=1,base=0)
        L = laplacian(G)
        r = spectralRad(L)
        broken = False
        for j in range(n):
            D = np.zeros((n,n))
            D[j,j] = 1
            if spectralRad(L+D)<r:
                broken = True
                break
        if broken:
            break
    return G,D
    

### Build graphs that we can specialize without problems
If we build a graph with one SCC outside the base (also a SCC) and only one link from the base to the component, most of the time, the spectral radius doen't increase upon specialization.

In [23]:
howMany = 1000
incr = 0
print("n\tPercent that are stable")
for n in range(3,7):
    for i in range(howMany):
        U,X = randomGraph(n,stronglyCon=1,base=0),np.zeros((n,n))
        i = np.random.randint(0,n)
        X[i,i] = 1
        S,Y = randomGraph(n,stronglyCon=1,base=0),np.zeros((n,n))
        i = np.random.randint(0,n)
        Y[i,i] = 1
        i = np.random.randint(0,n)
        X[i,i] = 1

        M = np.vstack((np.hstack((U,X)),np.hstack((Y,S))))
        sM = specializeGraph(M,[0,1,2,3])
        if spectralRad(laplacian(M)) <= spectralRad(laplacian(sM)) + 1e-8:
            incr += 1
    print("{}\t{}".format(2*n,float(incr)/howMany))
    incr=0

n	Percent that are stable
6	0.984
8	0.995
10	0.978
12	0.989


### Taking away from Diagonals
After I've added to a diagonal, can I take from a diagonal and always derease the spectral radius?

In [36]:
howMany = 1000
n=6
for k in range(howMany):
    
    S       = randomGraph(n,stronglyCon=1,base=0)
    i,j     = np.random.randint(0,n,size=2)
    D1      = np.zeros((n,n))
    D2      = np.zeros((n,n))
    D1[i,i] = 1
    D2[j,j] = 1
    rho     = spectralRad(laplacian(S+D1+D2))

    if spectralRad(laplacian(S+D1)) >= rho + 1e-8:
        print("D1 i = {}".format(k))
        break

    if spectralRad(laplacian(S+D2)) >= rho + 1e-8:
        print("D2 i = {}".format(k))
        break
        
    if k == 998:
        print("Success")


Success


It seems like you can